# Spam Vs Ham Classifier

This notebook corresponds to excercise #4 of the book, we will use the Apache Spam assasing corpus https://spamassassin.apache.org/old/publiccorpus/.

I'm going to work with:
* 20021010_spam.tar.bz2 (Contains emails flagged as spam)
* 20021010_easy_ham.tar.bz2 (Contains easily recognizable ham emails)
* 20021010_hard_ham.tar.bz2 (Contains ham emails but can be mistaken with spam)

## Data exploration
Let's see the content of these folders.


In [17]:
import os
import tarfile
from six.moves import urllib

DOWNLOAD_URL = "http://spamassassin.apache.org/old/publiccorpus/"
HAM_URL = DOWNLOAD_URL + "20021010_easy_ham.tar.bz2"
SPAM_URL = DOWNLOAD_URL + "20021010_spam.tar.bz2"
DATASETS_PATH = os.path.join('./', 'spamham')

def fetch_data(files, datasets_path=DATASETS_PATH):
    if not files:
        raise ValueError("You must provide the files to download")
    
    if type(files) != tuple:
        if type(files) == list and type(files[0]) != tuple:
            raise ValueError("The files must be a tuple or list of tuples e.g.: ('filename', 'url') or [('filename', 'url')]")
                
    if not os.path.isdir(datasets_path):
        os.makedirs(datasets_path)
        
    for filename, url in files:
        path = os.path.join(datasets_path, filename)
        if not os.path.isfile(path):
            urllib.request.urlretrieve(url, path)
        tar_bz2_file = tarfile.open(path)
        tar_bz2_file.extractall(path=DATASETS_PATH)
        tar_bz2_file.close()

In [18]:
files = [
    ('ham.tar.bz2', HAM_URL),
    ('spam.tar.bz2', SPAM_URL)
]

fetch_data(files)

In [16]:
type(files) == list and type(files[0]) != tuple

False